<a href="https://colab.research.google.com/github/EddyDavies/Trade_With_Twitter/blob/RL/ipynb/BERT_Base_Transformer_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    from google.colab import drive
    IN_COLAB = True
    drive.mount('/content/drive')
except ModuleNotFoundError:
    IN_COLAB = False

Mounted at /content/drive


Check GPU

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Sep 16 15:44:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Check TPU

Check Memory

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
!git clone https://github.com/EddyDavies/Trade_With_Twitter.git
!git pull

Cloning into 'Trade_With_Twitter'...
remote: Enumerating objects: 706, done.
remote: Counting objects: 100% (706/706), done.
remote: Compressing objects: 100% (483/483), done.
remote: Total 706 (delta 423), reused 447 (delta 215), pack-reused 0
Receiving objects: 100% (706/706), 176.72 KiB | 2.52 MiB/s, done.
Resolving deltas: 100% (423/423), done.
fatal: not a git repository (or any of the parent directories): .git


In [5]:
!ls ../

bin	 datalab  home	 lib64	opt	    root  srv		     tmp    var
boot	 dev	  lib	 media	proc	    run   sys		     tools
content  etc	  lib32  mnt	python-apt  sbin  tensorflow-1.15.2  usr


In [6]:
%cd Trade_With_Twitter/src
!ls

/content/Trade_With_Twitter/src
prices	rl  sentiment  trade.py  twitter  utils


In [7]:
!pip install -r ../requirements.txt

     |████████████████████████████████| 115 kB 14.9 MB/s 
     |████████████████████████████████| 2.8 MB 89.0 MB/s 
     |████████████████████████████████| 184 kB 95.6 MB/s 
     |████████████████████████████████| 6.3 MB 64.1 MB/s 
     |████████████████████████████████| 3.3 MB 75.0 MB/s 
     |████████████████████████████████| 895 kB 69.1 MB/s 
     |████████████████████████████████| 52 kB 2.1 MB/s 
     |████████████████████████████████| 636 kB 80.5 MB/s 
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=baee81eb6276cd552ce4b48e84a7e50dcbf446e0b3b0384d7bbf1983df2d6e6d
  Stored in directory: /root/.cache/pip/wheels/0b/81/f0/cca85757840e4616a2c6b9fe12569d97d324c27cac60724c58
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=639e7b4390e2885b988332d6d0ca9582530ea3506244cba243365634ae1ff01d
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
  Cre

In [11]:
import os
import pandas as pd
from tqdm import tqdm

from sentiment.inference import get_tweets, get_sentiments, save_sentiments
from sentiment.folder import get_paths
from utils.dates import get_date_array, get_month_array, check_last_day

In [12]:
dates_range = ("2017-01-01", "2021-05-31")
months_range = ["Jan 17", "Jun 21"]
# dates_range = ("2019-01-05", "2021-05-31")

months = get_month_array(months_range)
dates = get_date_array(dates_range)

In [13]:
import torch
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [14]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
model_name = "siebert/sentiment-roberta-large-english"

sentiment_analysis = pipeline("sentiment-analysis", model=model_name, device=0)

Downloading:   0%|          | 0.00/687 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [15]:
if IN_COLAB:
    source, results_folder = get_paths(data_folder='../../drive/MyDrive/ColabData', model_name=model_name)
else:
    source, results_folder = get_paths()
results_folder

'../../drive/MyDrive/ColabData/bitcoin_scores/sentiment-roberta-large-english_siebert'

In [ ]:
for date in dates:
    ids, tweets = get_tweets(date, source)

    percentage_per_chunk = 100
    save_every = 2000
    results = get_sentiments(date, tweets, ids, results_folder,
                              sentiment_analysis=sentiment_analysis,
                              save_every=save_every,
                              percentage_per_chunk=percentage_per_chunk,
                             slice_size=128,
                             batch_size=100)

    # check_last_day(results_folder, date)

2017-01-04:  37%|███▋      | 89/243 [00:47<01:20,  1.91it/s]

In [ ]:
!ls ../

In [ ]:
# !rm -r ../data/bitcoin_scores

In [ ]:
print(results_folder)

In [ ]:
!git pull